In [1]:
import pandas as pd
import numpy as np
import os

In [15]:
read_path=r'I:\PG\QInvestment\project\codesAndData'

In [16]:
paths=os.listdir(read_path)
readpaths=[]
for path in paths:
    if 'datagroup' in path:
        newpath=os.path.join(read_path,path)
        readpaths.append(newpath)       

In [18]:
def readcsv(readpaths):
    dfsum=pd.DataFrame(data=None)
    for path in readpaths:
        tempdata=pd.read_csv(path)
        dfsum=pd.concat([dfsum,tempdata])
    return dfsum

In [19]:
#整合所有股票所有时间的日内波动率
varianceDaily=readcsv(readpaths)
#varianceDaily.to_csv('I:\\PG\\QInvestment\\project\\varianceDaily.csv')

In [20]:
varianceDaily=(varianceDaily.iloc[:,1:]).reset_index(drop=True)

In [22]:
varianceDaily.groupby(['date']).count()

,code,variance
date,,
2017-10-16,943,943
2017-10-17,940,940
2017-10-18,939,939
2017-10-19,938,938
2017-10-20,937,937
...,...,...
2023-02-22,1000,1000
2023-02-23,1000,1000
2023-02-24,1000,1000


In [23]:
#计算重复
varianceDaily['duplicate']=varianceDaily.iloc[:,0:2].duplicated()
varianceDaily.iloc[:,0:2].duplicated().sum()

18933

In [24]:
#得到重复时间
##调整成分股的当天成分股与之前保持一致
dupdatelist=varianceDaily[varianceDaily['duplicate']==True]['date'].unique()

In [25]:
dupdatelist

array(['2018-06-11', '2018-07-13', '2018-12-17', '2019-06-17',
       '2019-12-16', '2020-01-13', '2020-12-14', '2020-02-17',
       '2020-06-15', '2020-07-13', '2020-08-17', '2020-10-12',
       '2021-12-13', '2021-02-18', '2021-03-15', '2021-06-15',
       '2021-09-13', '2021-10-11', '2022-02-14', '2022-06-13'],
      dtype=object)

In [26]:
#删除重复时间
#若先读入的保留，也就是删掉True
index_del=[]
boollist=[True,True,False,True,False,True,True,True,True,True,True,False,True,True,True,True,True,False,True,True]
for ii in range(len(dupdatelist)):
    #print(ii)
    temp_index=varianceDaily[(varianceDaily['date']==dupdatelist[ii]) &(varianceDaily['duplicate']==boollist[ii])].index.to_list()
    index_del+=temp_index

In [27]:
varianceDaily=varianceDaily.drop(index_del)

In [28]:
#计算VOV
varianceDaily['code']=varianceDaily['code'].apply(lambda x:x[3:])
finalData=varianceDaily.set_index(['date','code']).iloc[:,:-1].unstack()
finalData.columns=finalData.columns.droplevel(level=0)

In [29]:
VOVdata=finalData.apply(lambda x:x.dropna().rolling(window=40).std()).loc['2017-12-29':'2022-12-30',:]

In [30]:
VOVdata

code,000005,000006,000010,000011,000014,000016,000018,000019,000022,000025,...,688707,688711,688733,688739,688776,688778,688788,688789,688798,688800
date,,,,,,,,,,,,,,,,,,,,,
2017-12-29,NaN,NaN,1.040827,0.726552,NaN,1.599557,0.552014,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-02,NaN,NaN,1.040593,0.731812,NaN,1.616106,0.547723,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-03,NaN,NaN,1.016255,0.706903,NaN,1.586177,0.603891,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-04,NaN,NaN,1.006213,0.709437,NaN,1.584627,0.582763,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-05,NaN,NaN,1.005132,0.833095,NaN,1.496844,0.582876,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,NaN,2.224205,NaN,1.560265,NaN,0.222987,NaN,1.089849,NaN,1.211850,...,NaN,NaN,NaN,NaN,NaN,NaN,0.417038,NaN,NaN,NaN
2022-12-27,NaN,2.244545,NaN,1.545052,NaN,0.219764,NaN,1.084389,NaN,1.204863,...,NaN,NaN,NaN,NaN,NaN,NaN,0.407245,NaN,NaN,NaN
2022-12-28,NaN,2.584086,NaN,1.535340,NaN,0.218437,NaN,1.075592,NaN,1.215191,...,NaN,NaN,NaN,NaN,NaN,NaN,0.419592,NaN,NaN,NaN


In [31]:
#去掉2017-12-29到2022-12-30该段时间内无VOV值的股票
FinalVOV=VOVdata.dropna(axis=1,how='all')

In [34]:
FinalVOV.to_excel('风险模糊度.xlsx')

In [35]:
FinalVOV

code,000006,000010,000011,000016,000018,000019,000022,000025,000030,000032,...,688578,688586,688598,688608,688617,688680,688686,688696,688699,688788
date,,,,,,,,,,,,,,,,,,,,,
2017-12-29,NaN,1.040827,0.726552,1.599557,0.552014,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-02,NaN,1.040593,0.731812,1.616106,0.547723,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-03,NaN,1.016255,0.706903,1.586177,0.603891,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-04,NaN,1.006213,0.709437,1.584627,0.582763,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-05,NaN,1.005132,0.833095,1.496844,0.582876,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,2.224205,NaN,1.560265,0.222987,NaN,1.089849,NaN,1.211850,0.203204,1.906748,...,0.485035,0.392618,0.507838,0.892112,0.658034,0.792436,0.610353,0.608133,0.833919,0.417038
2022-12-27,2.244545,NaN,1.545052,0.219764,NaN,1.084389,NaN,1.204863,0.202826,1.915219,...,0.489607,0.368785,0.500463,0.891522,0.568176,0.817302,0.609797,0.602489,0.834078,0.407245
2022-12-28,2.584086,NaN,1.535340,0.218437,NaN,1.075592,NaN,1.215191,0.207336,1.923415,...,0.485447,0.320441,0.499934,0.881516,0.566979,0.809994,0.617526,0.607453,0.845056,0.419592


In [26]:
'''
#去掉和股票股价不匹配的两只股票
##匹配后手动求IC
###导入股票股价，和股票股价格式做匹配
pricedata=pd.read_csv(r'I:\PG\QInvestment\project\price.csv')
pricedata['sid']=pricedata['sid'].apply(lambda x: x[:-3])
pricedata=pricedata.set_index(['date','sid']).unstack()['close']
unfit=['000022','000043']
fitcode=[]
for code1 in FinalVOV.columns.to_list():
    if code1 not in unfit:
        fitcode.append(code1)
#fitcode=list(set(FinalVOV.columns.to_list())-unfit)
FinalVOV=FinalVOV.drop(unfit,axis=1)
FinalVOV.to_excel("风险模糊度.xlsx")
price_match=pricedata.loc[:,fitcode]
returndaily=price_match.apply(lambda x:x.pct_change(21).shift(1))
IC=price_match.corrwith(returndaily,axis=1).mean()
'''